In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.2 MB/s eta 0:00:00


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199676 sha256=54d2486598b8992d932ce501a83a90ab13a7d3e18aef7790cbfdeee27f78a4a2
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import json
from tqdm import tqdm
import pandas as pd
tqdm.pandas()
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding,Bidirectional,Concatenate
from keras.optimizers import *

from tensorflow.keras.utils import to_categorical

import pickle
from datasets import load_metric

from nltk.translate.bleu_score import corpus_bleu
import fasttext
import tqdm as notebook_tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read the training data from JSON file
train_data = pd.read_json("/content/drive/MyDrive/internship/asm_train.json", lines=True)
test_data=pd.read_json("/content/drive/MyDrive/internship/asm_test.json", lines=True)

In [ ]:
#converts the JSON data into a Pandas DataFrame
def correct_json_format(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        json_data = file.read()

    # Process the JSON string to insert commas between objects
    json_data = json_data.replace('}\n{', '},{')
    json_list = json.loads(f'[{json_data}]')
    df = pd.json_normalize(json_list)

    return df


In [ ]:
df_train = correct_json_format("/content/drive/MyDrive/internship/asm_train.json")
df_test = correct_json_format("/content/drive/MyDrive/internship/asm_test.json")


In [ ]:
df_train


,unique_identifier,native word,english word,source,score
0,asm1,দীপলিনা,deeplina,Wikidata,NaN
1,asm2,বন্দ্যোপাধ্যায়,bandyopadhyay,Wikidata,NaN
2,asm3,কৃষন,krishan,Wikidata,NaN
3,asm4,বিষয়া,bishaya,Wikidata,NaN
4,asm5,পেমা,pema,Wikidata,NaN
...,...,...,...,...,...
178625,asm178626,বাগিচাসমূহো,bagisaxomuhu,IndicCorp,-0.058495
178626,asm178627,কার্যনির্বাহক,karzonirbahok,IndicCorp,-0.222479
178627,asm178628,এইটি,eiti,IndicCorp,-0.161426
178628,asm178629,চিত্রশালাত,sitroxalat,IndicCorp,-0.111472


In [ ]:
df_test

,unique_identifier,native word,english word,source
0,asm1,লক্ষীনগৰস্থিত,lakhyeenogorsthito,AK-Freq
1,asm2,চতুৰ্থ,soturtho,AK-Freq
2,asm3,এইখন,eikhan,AK-Freq
3,asm4,প্ৰতিমূৰ্তিসমূহ,protimurtixomuh,AK-Freq
4,asm5,প্ৰতিযোগিতাতে,protijugitate,AK-Freq
...,...,...,...,...
5501,asm5502,ঝা,jha,AK-NEI
5502,asm5503,এলেকন,elecon,AK-NEI
5503,asm5504,ডেচাবাথুলা,desabathula,AK-NEI
5504,asm5505,কাৰখেলে,karkhele,AK-NEI


In [ ]:
#subset train DataFrame
subset_df_train=df_train.sample(frac=0.3).reset_index(drop=True)
subset_df_train.shape

(53589, 5)

In [ ]:
df_train=subset_df_train
df_train.shape

(53589, 5)

In [ ]:
 #start symbol "\t" at the beginning of the text and an end symbol "\n" at the end of the text(target_text)
def add_start_end(target_text):
  text = "\t" + target_text + "\n"
  return text


In [ ]:
#the DataFrame df_train will have a new column 'target_ass' containing the original text values from the 'native word' column with a start symbol "\t" at the beginning and an end symbol "\n" at the end of each text value.
df_train['target_ass'] = df_train['native word'].progress_apply(add_start_end)
df_train.head()

100%|██████████| 53589/53589 [00:00<00:00, 676302.79it/s]


,unique_identifier,native word,english word,source,score,target_ass
0,asm135203,দুৰ্ঘটনাবোৰে,durghotonabure,IndicCorp,-0.039397,\tদুৰ্ঘটনাবোৰে\n
1,asm7391,দলবিলাকেও,dolbilakeu,IndicCorp,-0.027320,\tদলবিলাকেও\n
2,asm102515,ছাত্ৰসকলো,satroxokolu,IndicCorp,-0.068606,\tছাত্ৰসকলো\n
3,asm87324,শিক্ষকেই,xikhyokei,IndicCorp,-0.014017,\tশিক্ষকেই\n
4,asm13219,গুণীৰ,gunir,IndicCorp,-0.113637,\tগুণীৰ\n


In [ ]:
# add only the end token
# add_end takes a string target_text as input and adds an end symbol "\n"
def add_end(target_text):
  text = target_text + "\n"
  return text

In [ ]:
 #adds a new column named 'decoder_target' to the DataFrame df_train. The values in this new column are derived from the values in the 'native word' column.
df_train['decoder_target'] = df_train['native word'].progress_apply(add_end)
df_train.head()


100%|██████████| 53589/53589 [00:00<00:00, 830535.15it/s]


,unique_identifier,native word,english word,source,score,target_ass,decoder_target
0,asm135203,দুৰ্ঘটনাবোৰে,durghotonabure,IndicCorp,-0.039397,\tদুৰ্ঘটনাবোৰে\n,দুৰ্ঘটনাবোৰে\n
1,asm7391,দলবিলাকেও,dolbilakeu,IndicCorp,-0.027320,\tদলবিলাকেও\n,দলবিলাকেও\n
2,asm102515,ছাত্ৰসকলো,satroxokolu,IndicCorp,-0.068606,\tছাত্ৰসকলো\n,ছাত্ৰসকলো\n
3,asm87324,শিক্ষকেই,xikhyokei,IndicCorp,-0.014017,\tশিক্ষকেই\n,শিক্ষকেই\n
4,asm13219,গুণীৰ,gunir,IndicCorp,-0.113637,\tগুণীৰ\n,গুণীৰ\n


In [ ]:
def tokenize(lang):
# Create tokenizer
    #(filters)means that all characters in the input text will be considered for tokenization,(char_lvl)each character will be treated as a separate token.
    tokenizer = Tokenizer(filters='',char_level=True)
    # Fit texts
     #creates the vocabulary based on the characters present in the corpus.
    tokenizer.fit_on_texts(lang)
    word_index = tokenizer.word_index
    return tokenizer,word_index

In [ ]:
# Tokenize words
roman_words = df_train['english word']
ass_words = df_train['target_ass']

roman_tokenizer, roman_tokens = tokenize(roman_words)
ass_tokenizer, ass_tokens = tokenize(ass_words)


print(roman_tokens)

print("No.of unique input tokens:", len(roman_tokens))


{'o': 1, 'a': 2, 'r': 3, 'i': 4, 't': 5, 'n': 6, 'u': 7, 'h': 8, 'k': 9, 'e': 10, 'b': 11, 'l': 12, 'm': 13, 'd': 14, 's': 15, 'p': 16, 'x': 17, 'g': 18, 'j': 19, 'y': 20, 'w': 21, 'c': 22, 'f': 23, 'v': 24, 'z': 25, 'q': 26}
No.of unique input tokens: 26


In [ ]:
roman_words[15]

'onubhutiburor'

In [ ]:
print(ass_tokens )
print("No.of unique output tokens:", len(ass_tokens))


{'\t': 1, '\n': 2, 'া': 3, 'ৰ': 4, '্': 5, 'ি': 6, 'ক': 7, 'ে': 8, 'ন': 9, 'ত': 10, 'ল': 11, 'ব': 12, 'ম': 13, 'প': 14, 'স': 15, 'ো': 16, 'ু': 17, 'দ': 18, 'ী': 19, 'ট': 20, 'হ': 21, 'জ': 22, 'গ': 23, 'ই': 24, 'চ': 25, 'য': 26, 'ৱ': 27, 'শ': 28, 'র': 29, 'ণ': 30, 'ভ': 31, 'খ': 32, 'ষ': 33, 'ধ': 34, 'ৈ': 35, 'ং': 36, 'ছ': 37, 'অ': 38, 'থ': 39, 'ড': 40, 'আ': 41, 'ূ': 42, 'ও': 43, 'ফ': 44, 'উ': 45, 'এ': 46, '়': 47, 'ৃ': 48, 'ঁ': 49, 'ঞ': 50, 'ঘ': 51, 'ঠ': 52, 'ঙ': 53, 'ৌ': 54, 'ৎ': 55, 'ঢ': 56, 'ঃ': 57, 'ঝ': 58, 'ঐ': 59, 'ঔ': 60, 'ঋ': 61, 'ঈ': 62, 'ঊ': 63}
No.of unique output tokens: 63


In [ ]:
# vocab size (no. of unique characters for each script)

roman_vocab = len(roman_tokens)+1
ass_vocab = len(ass_tokens)+1
roman_vocab,ass_vocab

(27, 64)

In [ ]:
roman_words[45],ass_words[45]

('aandul', '\tআন্দোল\n')

In [ ]:
# convert words to int sequence

roman_words_in_ids = roman_tokenizer.texts_to_sequences(roman_words)
ass_words_in_ids = ass_tokenizer.texts_to_sequences(ass_words)

In [ ]:
roman_words_in_ids[45],ass_words_in_ids[45]

([2, 2, 6, 14, 7, 12], [1, 41, 9, 5, 18, 16, 11, 2])

In [ ]:
roman_words_in_ids[5]

[12, 2, 13, 14, 4, 6, 18, 9, 8, 1, 6]

In [ ]:
roman_words[5]

'lamdingkhon'

In [ ]:
ass_words_in_ids[5]


[1, 11, 3, 13, 40, 6, 36, 32, 9, 2]

In [ ]:
# y_train
# only with end token

label_words = df_train["decoder_target"]
print(label_words[6])
#convert the text sequences in label_words into sequences of integers
y_train = ass_tokenizer.texts_to_sequences(label_words)
y_train

পলুৱাই



[[18, 17, 4, 5, 51, 20, 9, 3, 12, 16, 4, 8, 2],
 [18, 11, 12, 6, 11, 3, 7, 8, 43, 2],
 [37, 3, 10, 5, 4, 15, 7, 11, 16, 2],
 [28, 6, 7, 5, 33, 7, 8, 24, 2],
 [23, 17, 30, 19, 4, 2],
 [11, 3, 13, 40, 6, 36, 32, 9, 2],
 [14, 11, 17, 27, 3, 24, 2],
 [7, 11, 14, 17, 11, 6, 20, 16, 7, 2],
 [18, 8, 28, 15, 8, 27, 3, 4, 2],
 [14, 3, 10, 6, 4, 8, 24, 2],
 [32, 5, 29, 19, 33, 5, 20, 3, 12, 5, 18, 4, 2],
 [11, 16, 27, 3, 20, 16, 27, 8, 24, 2],
 [4, 36, 13, 36, 27, 8, 2],
 [44, 21, 6, 26, 47, 3, 24, 2],
 [13, 3, 21, 15, 13, 42, 21, 8, 24, 2],
 [38, 9, 17, 31, 42, 10, 6, 12, 16, 4, 4, 2],
 [23, 16, 20, 7, 8, 24, 20, 3, 2],
 [4, 17, 21, 17, 11, 4, 2],
 [7, 3, 52, 44, 17, 11, 3, 4, 2],
 [25, 4, 13, 14, 9, 5, 39, 19, 2],
 [18, 17, 32, 9, 21, 8, 2],
 [7, 5, 33, 9, 5, 10, 8, 7, 19, 26, 47, 3, 2],
 [15, 36, 32, 3, 2],
 [15, 13, 5, 14, 10, 5, 10, 6, 4, 16, 2],
 [45, 55, 15, 3, 10, 2],
 [31, 19, 13, 14, 4, 3, 2],
 [7, 52, 6, 10, 2],
 [7, 3, 10, 6, 12, 6, 21, 17, 4, 2],
 [13, 48, 18, 17, 2],
 [18, 6, 11, 3

In [ ]:
y_train[9]

[14, 3, 10, 6, 4, 8, 24, 2]

In [ ]:
print(label_words[9])

পাতিৰেই



In [ ]:
# max sequence length

max_encoder_seq_length_roman = df_train['english word'].str.len().max()
max_decoder_seq_length_ass = df_train['target_ass'].str.len().max()

max_encoder_seq_length_roman, max_decoder_seq_length_ass

(24, 21)

In [ ]:
def post_padding(data_in_int_seq, max_seq_len):
  return pad_sequences(data_in_int_seq, max_seq_len, padding = "post")
# padding

roman_padded = post_padding(roman_words_in_ids, max_encoder_seq_length_roman)
ass_padded = post_padding(ass_words_in_ids, max_decoder_seq_length_ass)
y_train_padded = post_padding(y_train, max_decoder_seq_length_ass)

print(roman_padded[16], len(roman_padded[16]))
print(ass_padded[16], len(ass_padded[16]))
print(y_train_padded[16],len(y_train_padded[16]))

[18  7  5  9 10  4  5  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0] 24
[ 1 23 16 20  7  8 24 20  3  2  0  0  0  0  0  0  0  0  0  0  0] 21
[23 16 20  7  8 24 20  3  2  0  0  0  0  0  0  0  0  0  0  0  0] 21


In [ ]:
latent_dim = 64
emb_dim = 100

In [ ]:
# Define the encoder model
encoder_inputs = Input(shape=(max_encoder_seq_length_roman,), name="encoder_input")
emb_encoder = Embedding(roman_vocab, emb_dim, mask_zero=True, name="encoder_embedding")(encoder_inputs)
#encoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=False,return_state=True, name="encoder_bilstm_1"))(emb_encoder)
encoder_outputs, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(latent_dim, return_state=True, return_sequences=False),name="BiLSTM",merge_mode="mul")(emb_encoder)


In [ ]:
# concatenate along the last dimension
encoder_h = Concatenate(axis=-1,name="concat_h")([forward_h, backward_h])
encoder_c = Concatenate(axis=-1,name="concat_c")([forward_c, backward_c])
encoder_h, encoder_c


(<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_h')>,
 <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_c')>)

In [ ]:
encoder_states = [encoder_h,encoder_c]
encoder_states

[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_h')>,
 <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_c')>]

In [ ]:
# decoder part

decoder_inputs = keras.Input(shape=(None,),name="decoder_input")

decoder_emb = Embedding(ass_vocab, emb_dim, mask_zero=True, name="decoder_embedding")(decoder_inputs)
d_lstm_out,d_h,d_c = LSTM(128, return_state=True, return_sequences=True, name="decoder_lstm_1")(decoder_emb, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(ass_vocab, activation="softmax")

decoder_outputs = decoder_dense(d_lstm_out)

In [ ]:
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9)

model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 24)]         0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 24, 100)      2700        ['encoder_input[0][0]']          
                                                                                                  
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 BiLSTM (Bidirectional)         [(None, 64),         84480       ['encoder_embedding[0][0]']      
                                 (None, 64),                                                  

In [ ]:
roman_padded[45]

array([ 2,  2,  6, 14,  7, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [ ]:
roman_words[45]

'aandul'

In [ ]:

ass_padded[45]

array([ 1, 41,  9,  5, 18, 16, 11,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)

In [ ]:
y_train_padded[53587]

array([ 4, 17, 15,  5, 10, 13,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)

In [ ]:
batch_size = 64
epochs = 100


model.fit(
    [roman_padded, ass_padded],
    y_train_padded,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
model.save("/content/drive/MyDrive/internship/mt_bilstm_1l.h5")

Epoch 1/100
670/670 [==============================] - 42s 34ms/step - loss: 1.0971 - accuracy: 0.6896 - val_loss: 0.4396 - val_accuracy: 0.8608
Epoch 2/100
670/670 [==============================] - 16s 23ms/step - loss: 0.3714 - accuracy: 0.8789 - val_loss: 0.3329 - val_accuracy: 0.8897
Epoch 3/100
670/670 [==============================] - 15s 22ms/step - loss: 0.2874 - accuracy: 0.9029 - val_loss: 0.2896 - val_accuracy: 0.9045
Epoch 4/100
670/670 [==============================] - 17s 25ms/step - loss: 0.2478 - accuracy: 0.9148 - val_loss: 0.2792 - val_accuracy: 0.9054
Epoch 5/100
670/670 [==============================] - 16s 24ms/step - loss: 0.2255 - accuracy: 0.9216 - val_loss: 0.2510 - val_accuracy: 0.9150
Epoch 6/100
670/670 [==============================] - 15s 23ms/step - loss: 0.2077 - accuracy: 0.9273 - val_loss: 0.2512 - val_accuracy: 0.9156
Epoch 7/100
670/670 [==============================] - 15s 22ms/step - loss: 0.1952 - accuracy: 0.9314 - val_loss: 0.2466 - val_ac

In [ ]:
#Load saved model
saved_model = tf.keras.models.load_model('/content/drive/MyDrive/internship/mt_bilstm_1l.h5')
saved_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 24)]         0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 24, 100)      2700        ['encoder_input[0][0]']          
                                                                                                  
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 BiLSTM (Bidirectional)         [(None, 64),         84480       ['encoder_embedding[0][0]']      
                                 (None, 64),                                                  

In [ ]:
inputs = saved_model.get_layer('encoder_input').output                    #Encoder input
bi_lstm_out,f_h,f_c,b_h,b_c = saved_model.get_layer('BiLSTM').output   #Encoder LSTM output

concate_h = saved_model.get_layer('concat_h').output
concate_c = saved_model.get_layer('concat_c').output


In [ ]:
bi_lstm_out

<KerasTensor: shape=(None, 64) dtype=float32 (created by layer 'BiLSTM')>

In [ ]:

targets = saved_model.get_layer('decoder_input').output           #Decoder input
embedding_layer = saved_model.get_layer('decoder_embedding')      #Decoder Embedding layer
decoder_lstm1 = saved_model.get_layer('decoder_lstm_1')           #Decoder LSTM layer
dense1 = saved_model.get_layer('dense')


In [ ]:
#Encoder

encoder = keras.Model(inputs, [concate_h, concate_c])
encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 24)]         0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 24, 100)      2700        ['encoder_input[0][0]']          
                                                                                                  
 BiLSTM (Bidirectional)         [(None, 64),         84480       ['encoder_embedding[0][0]']      
                                 (None, 64),                                                      
                                 (None, 64),                                                      
                                 (None, 64),                                                

In [ ]:
#Decoder

decoder_input_h1 = Input(shape=(128,))
decoder_input_c1 = Input(shape=(128,))

x = embedding_layer(targets)


In [ ]:
x

<KerasTensor: shape=(None, None, 100) dtype=float32 (created by layer 'decoder_embedding')>

In [ ]:
x,d_out_h,d_out_c = decoder_lstm1(x, initial_state=[decoder_input_h1, decoder_input_c1])

In [ ]:
#x = decoder_lstm2(x, initial_state=[decoder_input_h1, decoder_input_c1])

x = dense1(x)
x.shape


TensorShape([None, None, 64])

In [ ]:
decoder = keras.Model([targets]+[decoder_input_h1, decoder_input_c1] , [x] + [d_out_h, d_out_c])

decoder.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 decoder_embedding (Embedding)  (None, None, 100)    6400        ['decoder_input[0][0]']          
                                                                                                  
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                            

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in roman_tokens.items())
reverse_target_char_index = dict((i, char) for char, i in ass_tokens.items())

In [ ]:
reverse_input_char_index

{1: 'o',
 2: 'a',
 3: 'r',
 4: 'i',
 5: 't',
 6: 'n',
 7: 'u',
 8: 'h',
 9: 'k',
 10: 'e',
 11: 'b',
 12: 'l',
 13: 'm',
 14: 'd',
 15: 's',
 16: 'p',
 17: 'x',
 18: 'g',
 19: 'j',
 20: 'y',
 21: 'w',
 22: 'c',
 23: 'f',
 24: 'v',
 25: 'z',
 26: 'q'}

In [ ]:
reverse_target_char_index

{1: '\t',
 2: '\n',
 3: 'া',
 4: 'ৰ',
 5: '্',
 6: 'ি',
 7: 'ক',
 8: 'ে',
 9: 'ন',
 10: 'ত',
 11: 'ল',
 12: 'ব',
 13: 'ম',
 14: 'প',
 15: 'স',
 16: 'ো',
 17: 'ু',
 18: 'দ',
 19: 'ী',
 20: 'ট',
 21: 'হ',
 22: 'জ',
 23: 'গ',
 24: 'ই',
 25: 'চ',
 26: 'য',
 27: 'ৱ',
 28: 'শ',
 29: 'র',
 30: 'ণ',
 31: 'ভ',
 32: 'খ',
 33: 'ষ',
 34: 'ধ',
 35: 'ৈ',
 36: 'ং',
 37: 'ছ',
 38: 'অ',
 39: 'থ',
 40: 'ড',
 41: 'আ',
 42: 'ূ',
 43: 'ও',
 44: 'ফ',
 45: 'উ',
 46: 'এ',
 47: '়',
 48: 'ৃ',
 49: 'ঁ',
 50: 'ঞ',
 51: 'ঘ',
 52: 'ঠ',
 53: 'ঙ',
 54: 'ৌ',
 55: 'ৎ',
 56: 'ঢ',
 57: 'ঃ',
 58: 'ঝ',
 59: 'ঐ',
 60: 'ঔ',
 61: 'ঋ',
 62: 'ঈ',
 63: 'ঊ'}

In [ ]:
v = np.reshape(roman_padded[4], (1, -1))


In [ ]:
encoder.predict(v,verbose=0)

[array([[ 1.33771362e-04,  7.90653527e-01, -1.04795262e-01,
         -2.91916966e-01,  9.45403516e-01,  2.11774968e-02,
          7.85831094e-01, -7.65107632e-01,  4.72740322e-01,
         -2.04221974e-03, -9.31629479e-01, -2.66142666e-01,
          4.49504936e-03,  1.56077265e-03,  7.81473875e-01,
         -8.34877014e-01, -1.99113071e-01, -1.44018530e-04,
          1.25540580e-04, -2.23382656e-02,  4.12543659e-06,
          9.63540673e-01, -6.32729113e-01,  8.04393707e-08,
          3.28088561e-07, -9.79858334e-04,  1.61823682e-05,
         -3.88157144e-02, -2.15157121e-02,  2.91703105e-01,
          6.71329617e-05, -1.24633238e-01, -4.69755381e-02,
          1.50915412e-06, -4.94650430e-06,  8.04238617e-01,
          8.88554860e-06, -4.31182925e-05,  1.70787505e-04,
          3.57640892e-01, -8.02766755e-02, -7.62621403e-01,
          5.01430511e-01,  2.12626979e-01,  1.29222095e-01,
         -1.97254613e-01, -2.48747785e-02, -3.84415716e-06,
          6.05187836e-07,  3.13164815e-0

In [ ]:
def predict_list_of_words(list_source_words_in_padded_int_seq):
  list_pred_words = []
  start_token = np.zeros((1, 1))
  start_token[0] = ass_tokenizer.word_index['\t']
  print(start_token)

  for x in tqdm(range(len(list_source_words_in_padded_int_seq))):
    source_seq = start_token
    pred_word = ''
    v = np.reshape(list_source_words_in_padded_int_seq[x], (1, -1))
    next_h, next_c = encoder.predict(v,verbose=0)
    for i in range(max_decoder_seq_length_ass):
      output, next_h, next_c = decoder.predict([source_seq] + [next_h, next_c],verbose=0)
      next_token = np.argmax(output[0, 0, :])
      next_char = reverse_target_char_index[next_token]
      if next_char == '\n':
        break
      else:
        pred_word += next_char
        source_seq = np.zeros((1, 1))
        source_seq[0] = next_token
    list_pred_words.append(pred_word)
  return list_pred_words

In [ ]:
predicted = predict_list_of_words(roman_padded[:20])

# Print the predicted results
print(predicted)


[[1.]]


100%|██████████| 20/20 [00:15<00:00,  1.31it/s]

['দুৰ্ঘটনাবোৰে', 'দলবিলাকেও', 'ছাত্রসকলো', 'শিক্ষকেই', 'গুণীৰ', 'লামদিংখন', 'পলুৱাই', 'কলপুলিটোক', 'দেশসেৱাৰ', 'পাতিৰেই', 'খ্ৰীষ্টাব্দৰ', 'লোৱাটোৱেই', 'ৰংমংৱে', 'ফহিয়াই', 'মাহ্যসমূহে', 'অনুভূতিবোৰৰ', 'গোটকেইটা', 'ৰোহুলৰ', 'কাঠফুলাৰ', 'চৰমপন্থী']


In [ ]:
predicted = predict_list_of_words(roman_padded[:20])

[[1.]]


100%|██████████| 20/20 [00:13<00:00,  1.51it/s]


In [ ]:
df_analyse = pd.DataFrame()
df_analyse["source"] = roman_words[:20]
df_analyse["predicted"] = predicted
df_analyse["ground_truth"] = df_train["native word"]

df_analyse

,source,predicted,ground_truth
0,durghotonabure,দুৰ্ঘটনাবোৰে,দুৰ্ঘটনাবোৰে
1,dolbilakeu,দলবিলাকেও,দলবিলাকেও
2,satroxokolu,ছাত্রসকলো,ছাত্ৰসকলো
3,xikhyokei,শিক্ষকেই,শিক্ষকেই
4,gunir,গুণীৰ,গুণীৰ
5,lamdingkhon,লামদিংখন,লামডিংখন
6,poluwai,পলুৱাই,পলুৱাই
7,kolpulituk,কলপুলিটোক,কলপুলিটোক
8,dexxewar,দেশসেৱাৰ,দেশসেৱাৰ
9,patirei,পাতিৰেই,পাতিৰেই


In [ ]:
test_roman_words = df_test['english word']
test_target_words = df_test['native word']
test_roman_words[2],test_target_words[2]

('eikhan', 'এইখন')

In [ ]:
test_roman_words_in_ids = roman_tokenizer.texts_to_sequences(test_roman_words)
test_target_words_in_ids = ass_tokenizer.texts_to_sequences(test_target_words)

print(test_roman_words_in_ids[4])
print(test_target_words_in_ids[4])

[16, 3, 1, 5, 4, 19, 7, 18, 4, 5, 2, 5, 10]
[14, 5, 4, 10, 6, 26, 16, 23, 6, 10, 3, 10, 8]


In [ ]:
#2. padding

test_roman_padded = post_padding(test_roman_words_in_ids, max_encoder_seq_length_roman)
test_target_padded = post_padding(test_target_words_in_ids, max_decoder_seq_length_ass)

print(test_roman_padded[2])
print(test_target_padded[2])

[10  4  9  8  2  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[46 24 32  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [ ]:
len(test_roman_padded[2]), len(test_target_padded[2])

(24, 21)

In [ ]:
# prediction on test set

predicted_test = predict_list_of_words(test_roman_padded[:1500])

[[1.]]


100%|██████████| 1500/1500 [16:27<00:00,  1.52it/s]


In [ ]:
# analyse

df_analyse_test = pd.DataFrame()
df_analyse_test["source"] = test_roman_words[:1500]
df_analyse_test["predicted"] = predicted_test
df_analyse_test["ground_truth_label"] = df_test["native word"][:1500]

df_analyse_test.head(10)

,source,predicted,ground_truth_label
0,lakhyeenogorsthito,লেখ্যাংগৰ্শ্যতিৰহে,লক্ষীনগৰস্থিত
1,soturtho,চতুৰ্থ,চতুৰ্থ
2,eikhan,এইখান,এইখন
3,protimurtixomuh,প্ৰতিমূৰ্তিসমূহ,প্ৰতিমূৰ্তিসমূহ
4,protijugitate,প্ৰতিযোগিতাতে,প্ৰতিযোগিতাতে
5,niya,নিয়া,নিয়া
6,aason,আছন,আঁচন
7,deutaloi,দেউতালৈ,দেউতালৈ
8,eaglenest,ইগেন্টেন্স,ঈগলনেষ্ট
9,xeehotok,শিহঁতৰ,সিহঁতক


In [ ]:
# Calculate accuracy

accuracy = (df_analyse_test['predicted'] == df_analyse_test['ground_truth_label']).mean()

print(f'Test Accuracy = {accuracy*100:.2f}%')

Test Accuracy = 36.73%


In [ ]:

def split_characters(text_string):
  return [character for character in text_string]

split_characters('চ' 'তু' 'ৰ্থ')

['চ', 'ত', 'ু', 'ৰ', '্', 'থ']

In [ ]:
df_analyse_test["predicted_tokenized"] = df_analyse_test["predicted"].apply(split_characters)
df_analyse_test["label_tokenized"] = df_analyse_test["ground_truth_label"].apply(split_characters)

df_analyse_test.head()

,source,predicted,ground_truth_label,predicted_tokenized,label_tokenized
0,lakhyeenogorsthito,লেখ্যাংগৰ্শ্যতিৰহে,লক্ষীনগৰস্থিত,"[ল, ে, খ, ্, য, া, ং, গ, ৰ, ্, শ, ্, য, ত, ি, ...","[ল, ক, ্, ষ, ী, ন, গ, ৰ, স, ্, থ, ি, ত]"
1,soturtho,চতুৰ্থ,চতুৰ্থ,"[চ, ত, ু, ৰ, ্, থ]","[চ, ত, ু, ৰ, ্, থ]"
2,eikhan,এইখান,এইখন,"[এ, ই, খ, া, ন]","[এ, ই, খ, ন]"
3,protimurtixomuh,প্ৰতিমূৰ্তিসমূহ,প্ৰতিমূৰ্তিসমূহ,"[প, ্, ৰ, ত, ি, ম, ূ, ৰ, ্, ত, ি, স, ম, ূ, হ]","[প, ্, ৰ, ত, ি, ম, ূ, ৰ, ্, ত, ি, স, ম, ূ, হ]"
4,protijugitate,প্ৰতিযোগিতাতে,প্ৰতিযোগিতাতে,"[প, ্, ৰ, ত, ি, য, ো, গ, ি, ত, া, ত, ে]","[প, ্, ৰ, ত, ি, য, ো, গ, ি, ত, া, ত, ে]"


In [ ]:
def create_reference(tokenized):
  return [tokenized]

df_analyse_test["reference"] = df_analyse_test["label_tokenized"].apply(create_reference)
df_analyse_test.head()

,source,predicted,ground_truth_label,predicted_tokenized,label_tokenized,reference
0,lakhyeenogorsthito,লেখ্যাংগৰ্শ্যতিৰহে,লক্ষীনগৰস্থিত,"[ল, ে, খ, ্, য, া, ং, গ, ৰ, ্, শ, ্, য, ত, ি, ...","[ল, ক, ্, ষ, ী, ন, গ, ৰ, স, ্, থ, ি, ত]","[[ল, ক, ্, ষ, ী, ন, গ, ৰ, স, ্, থ, ি, ত]]"
1,soturtho,চতুৰ্থ,চতুৰ্থ,"[চ, ত, ু, ৰ, ্, থ]","[চ, ত, ু, ৰ, ্, থ]","[[চ, ত, ু, ৰ, ্, থ]]"
2,eikhan,এইখান,এইখন,"[এ, ই, খ, া, ন]","[এ, ই, খ, ন]","[[এ, ই, খ, ন]]"
3,protimurtixomuh,প্ৰতিমূৰ্তিসমূহ,প্ৰতিমূৰ্তিসমূহ,"[প, ্, ৰ, ত, ি, ম, ূ, ৰ, ্, ত, ি, স, ম, ূ, হ]","[প, ্, ৰ, ত, ি, ম, ূ, ৰ, ্, ত, ি, স, ম, ূ, হ]","[[প, ্, ৰ, ত, ি, ম, ূ, ৰ, ্, ত, ি, স, ম, ূ, হ]]"
4,protijugitate,প্ৰতিযোগিতাতে,প্ৰতিযোগিতাতে,"[প, ্, ৰ, ত, ি, য, ো, গ, ি, ত, া, ত, ে]","[প, ্, ৰ, ত, ি, য, ো, গ, ি, ত, া, ত, ে]","[[প, ্, ৰ, ত, ি, য, ো, গ, ি, ত, া, ত, ে]]"


In [ ]:
# BLEU Score

# Convert DataFrame columns to lists
predictions = df_analyse_test['predicted_tokenized'].tolist()
true_labels = df_analyse_test['reference'].tolist()


In [ ]:
true_labels

[[['ল', 'ক', '্', 'ষ', 'ী', 'ন', 'গ', 'ৰ', 'স', '্', 'থ', 'ি', 'ত']],
 [['চ', 'ত', 'ু', 'ৰ', '্', 'থ']],
 [['এ', 'ই', 'খ', 'ন']],
 [['প', '্', 'ৰ', 'ত', 'ি', 'ম', 'ূ', 'ৰ', '্', 'ত', 'ি', 'স', 'ম', 'ূ', 'হ']],
 [['প', '্', 'ৰ', 'ত', 'ি', 'য', 'ো', 'গ', 'ি', 'ত', 'া', 'ত', 'ে']],
 [['ন', 'ি', 'য', '়', 'া']],
 [['আ', 'ঁ', 'চ', 'ন']],
 [['দ', 'ে', 'উ', 'ত', 'া', 'ল', 'ৈ']],
 [['ঈ', 'গ', 'ল', 'ন', 'ে', 'ষ', '্', 'ট']],
 [['স', 'ি', 'হ', 'ঁ', 'ত', 'ক']],
 [['প', 'ূ', 'র', '্', 'ব', 'া', 'ঞ', '্', 'চ', 'ল', 'জ', 'ু', 'ৰ', 'ি']],
 [['প', 'ৰ', 'ি', 'দ', 'ৰ', '্', 'শ', 'ক']],
 [['হ', 'ে', 'ৰ', 'ু', 'ৱ', 'া', 'ই', 'ছ', 'ি', 'ল', 'ো']],
 [['স', 'দ', 'স', '্', 'য', 'স', 'ক', 'ল', 'ে']],
 [['স', 'ং', 'ক', '্', 'র', 'া', 'ন', '্', 'ত', 'ি', 'ৰ']],
 [['শ', 'ি', 'ক', '্', 'ষ', 'া', 'গ', 'ত', 'ভ', 'া', 'ৱ', 'ে']],
 [['ক', 'ৰ', 'ি', 'ল', 'ো']],
 [['ত', 'চ', 'ে', 'ন']],
 [['গ', 'ো', 'ৰ', '্', 'খ', 'া', 'স', 'ক', 'ল', 'ক']],
 [['ছ', 'া', 'ত', '্', 'ৰ', 'গ', 'ৰ', 'া', 'ক', 'ী', 'ক']],
 [['প', '্', 'ৰ', 'ত

In [ ]:
predictions

[['ল',
  'ে',
  'খ',
  '্',
  'য',
  'া',
  'ং',
  'গ',
  'ৰ',
  '্',
  'শ',
  '্',
  'য',
  'ত',
  'ি',
  'ৰ',
  'হ',
  'ে'],
 ['চ', 'ত', 'ু', 'ৰ', '্', 'থ'],
 ['এ', 'ই', 'খ', 'া', 'ন'],
 ['প', '্', 'ৰ', 'ত', 'ি', 'ম', 'ূ', 'ৰ', '্', 'ত', 'ি', 'স', 'ম', 'ূ', 'হ'],
 ['প', '্', 'ৰ', 'ত', 'ি', 'য', 'ো', 'গ', 'ি', 'ত', 'া', 'ত', 'ে'],
 ['ন', 'ি', 'য', '়', 'া'],
 ['আ', 'ছ', 'ন'],
 ['দ', 'ে', 'উ', 'ত', 'া', 'ল', 'ৈ'],
 ['ই', 'গ', 'ে', 'ন', '্', 'ট', 'ে', 'ন', '্', 'স'],
 ['শ', 'ি', 'হ', 'ঁ', 'ত', 'ৰ'],
 ['প', 'ৰ', '্', 'ব', 'া', 'চ', 'ঞ', 'ল', 'ু', 'ব', 'া', 'ই'],
 ['প', 'ৰ', 'ি', 'দ', 'র', '্', 'ষ', 'ক'],
 ['হ', 'ে', 'ৰ', 'ু', 'ৱ', 'া', 'ই', 'ছ', 'ি', 'ল', 'ো', 'ঁ'],
 ['চ', 'ধ', 'স', '্', 'য', 'ক', 'ল', 'ে', 'স', 'ৰ'],
 ['স', 'ং', 'ক', '্', 'ৰ', 'া', 'ন', '্', 'ত', 'ি', 'ৰ'],
 ['শ', 'ি', 'ক', '্', 'ষ', 'া', 'ত', 'ভ', 'া', 'ৱ', 'ে', 'দ'],
 ['ক', 'ৰ', 'ি', 'ল', 'ো'],
 ['ট', 'ছ', 'ে', 'ন'],
 ['গ', 'ু', 'ৰ', 'খ', 'া', 'খ', 'ল', 'ক', 'ঃ'],
 ['ছ', 'া', 'ত', '্', 'র', 'গ', 'ৰ', 'া', 'ক', 'ী', 'ক

In [ ]:
bleu = load_metric("bleu")
bleu.compute(predictions = predictions, references = true_labels)


<ipython-input-76-29ca3862bcb2>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu = load_metric("bleu")


{'bleu': 0.635469968852037,
 'precisions': [0.8437915006640107,
  0.6742510428517254,
  0.5737179487179487,
  0.4996028594122319],
 'brevity_penalty': 1.0,
 'length_ratio': 1.013373706787787,
 'translation_length': 12048,
 'reference_length': 11889}

In [ ]:
def transliterate(source_word_in_padded_int_seq):
  #list_pred_words = []
  start_token = np.zeros((1, 1))
  start_token[0] = ass_tokenizer.word_index['\t']
  #print(start_token)

  #for x in tqdm(range(len(list_source_words_in_padded_int_seq))):
  source_seq = start_token
  pred_word = ''
  #v = np.reshape(source_word_in_padded_int_seq, (1, -1))
  next_h, next_c = encoder.predict(source_word_in_padded_int_seq,verbose=0)
  for i in range(max_decoder_seq_length_ass):
    output, next_h, next_c = decoder.predict([source_seq] + [next_h, next_c],verbose=0)
    next_token = np.argmax(output[0, 0, :])
    next_char = reverse_target_char_index[next_token]
    if next_char == '\n':
      break
    else:
      pred_word += next_char
      source_seq = np.zeros((1, 1))
      source_seq[0] = next_token
  #list_pred_words.append(pred_word)
  return pred_word

input_word = input("\nEnter a romanized word: ")
input_word_in_ids = roman_tokenizer.texts_to_sequences([input_word])

input_word_in_ids_padded = post_padding(input_word_in_ids, max_encoder_seq_length_roman)

print(f"\nTransliterated word : {transliterate(input_word_in_ids_padded)}")



Enter a romanized word: moi

Transliterated word : মই
